In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
import sys
from pathlib import Path

parent = str(Path().absolute().parent)
if parent not in sys.path:
    sys.path.insert(0,parent)
from sherlock.evidence_store import EvidenceStore, GmailEvidenceStore
from sherlock.models import Claim, Evidence, Argument, EvidenceCollection
from sherlock.agents import ClaimInvestigationAgent
from sherlock.utils import export_argdown
from sherlock.gmail import create_service
import pyperclip

In [ ]:
store = EvidenceStore("wally_evidence")

# Add our evidence
evidence_ids = []
evidence_ids.append(store.add_evidence("Librarian Jane saw someone matching Wally's description at 3pm"))
evidence_ids.append(store.add_evidence("Security camera shows someone in red and white stripes in the park"))
evidence_ids.append(store.add_evidence("Library card records don't show Wally checking out any books today"))
evidence_ids.append(store.add_evidence("Wally's phone GPS shows he was at the library coordinates"))
evidence_ids.append(store.add_evidence("Wally always wears a read and white striped top"))



In [ ]:


agent_pro = ClaimInvestigationAgent(store, supports=True)
agent_con = ClaimInvestigationAgent(store, supports=False)
# Create a test claim
wally_claim = Claim(text="Wally is in the library")


In [ ]:
wally_claim = agent_pro.evaluate_claim(wally_claim)
wally_claim = agent_con.evaluate_claim(wally_claim)
# Evaluate it
wally_claim.likelihood

In [ ]:
arg = export_argdown(wally_claim)

In [ ]:
with open('argdown.txt', 'w') as f:
    f.write(arg)

In [ ]:
wally_park_claim = Claim(text="Wally is in the park")
wally_park_claim = agent_pro.evaluate_claim(wally_park_claim)
wally_park_claim = agent_con.evaluate_claim(wally_park_claim)
arg_park = export_argdown(wally_park_claim)
with open('argdown_park.txt', 'w') as f:
    f.write(arg_park)

In [ ]:
agent_pro.query_evidence('wally park')

In [ ]:
# Initialize Gmail evidence store (will prompt for authentication on first use)
try:
    gmail_store = GmailEvidenceStore()
    print("Gmail store initialized successfully!")
except FileNotFoundError as e:
    print(f"Gmail setup needed: {e}")
    gmail_store = None


In [ ]:
# Test Gmail search (only if gmail_store is available)
if gmail_store:
    # Search for recent emails about meetings
    email_results = gmail_store.query("booking AROUND hotel after:2025-07-01")
    print(f"Found {len(email_results)} email results")
    
    # Show first result
    if email_results:
        print("\nFirst email result:")
        print(f"Snippet: {email_results[0].get('snippet', '')}")
        print(f"Text preview: {email_results[0]['text'][:200]}...")
        if 'metadata' in email_results[0]:
            print(f"Subject: {email_results[0]['metadata']['subject']}")
            print(f"From: {email_results[0]['metadata']['sender']}")
else:
    print("Gmail store not available. Please set up credentials first.")


In [ ]:
if gmail_store:
    # Create agents that use Gmail as evidence source
    gmail_agent_pro = ClaimInvestigationAgent(gmail_store, supports=True, max_iterations=10)
    gmail_agent_con = ClaimInvestigationAgent(gmail_store, supports=False, max_iterations=10)
    
    # Example claim about your email habits
    email_claim = Claim(text="Andy travelled abroad in 2025")
    
    print("Evaluating claim using Gmail evidence...")
    # Note: This will search your actual Gmail for evidence
    email_claim = gmail_agent_pro.evaluate_claim(email_claim)
    #email_claim = gmail_agent_con.evaluate_claim(email_claim)
    
    # print(f"Claim likelihood: {email_claim.likelihood}")

else:
    print("Gmail store not available for agent testing.")


In [ ]:
email_claim.model_dump()

In [ ]:
# Create a claim that might benefit from multiple evidence sources
complex_claim = Claim(text="The project meeting was productive")

print("This claim could be evaluated using:")
print("1. ChromaDB store: Project documents, notes, reports")
print("2. Gmail store: Meeting invites, follow-up emails, action items")
print("\nEach agent can specialize in different types of evidence!")
